# Module 8: Time Series Analysis for Stock Price Prediction

__Key Takeaways__:
By the end of this module, students will gain a comprehensive understanding of time series analysis techniques and their application in forecasting stock prices. They will be equipped with practical skills to build and evaluate time series models using real-world financial data.

## Libraries

In [4]:
import pandas as pd
import numpy as np

# import yfinance as yf
import sqlite3

# import pytimetk as tk
# import skimpy as sk

# import pycaret
# from pycaret.time_series import *

# from sktime.forecasting.arima import AutoARIMA
# from sktime.forecasting.base import ForecastingHorizon

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Extract Stocks Data

In [2]:
start_date = '2023-01-01'
end_date = '2023-12-31'
tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']

def extract_data(tickers, start=start_date, end=end_date):
    df = pd.DataFrame()
    for ticker in tickers:
        current_df = yf.download(ticker, start=start_date, end=end_date).assign(Symbol = ticker)
        df = pd.concat([df, current_df])
    
    df = df .reset_index()
    
    # Convert column names to lowercase
    df.columns = (
        df.columns
        .str.lower()
        .str.replace(' ', '_')
    )

    return df

# Test the function
df = extract_data(tickers, start=start_date, end=end_date)
df.head()

NameError: name 'yf' is not defined

## Data Profiling

In [ ]:
df = pd.read_csv('./data/stocks.csv')
df.head()

,Date,Price
0,2020-01-02,154.493835
1,2020-01-03,152.570099
2,2020-01-06,152.964462
3,2020-01-07,151.569778
4,2020-01-08,153.984055


In [ ]:
sk.skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1000   │ │ float64     │ 5     │                                                          │
│ │ Number of columns │ 8      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 1     │                                                          │
│                                │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_n ┃    ┃      ┃          ┃          ┃          ┃          ┃          ┃          ┃          ┃        ┃  │
│ ┃ ame      ┃ NA ┃ NA % ┃ mean     ┃ sd       ┃ p0       ┃ p25      ┃ p50      ┃ p75      ┃ p100     ┃ hist   ┃  │
│ ┡━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩  │
│ │ open     │  0 │    0 │      210 │       78 │       86 │      140 │      190 │      260 │      380 │ ▇▇▅▅▃▃ │  │
│ │ high     │  0 │    0 │      210 │       79 │       88 │      140 │      190 │      260 │      380 │ ▇▇▅▆▃▃ │  │
│ │ low      │  0 │    0 │      200 │       78 │       85 │      140 │      180 │      260 │      380 │ ▇▇▅▅▃▃ │  │
│ │ close    │  0 │    0 │      210 │       78 │       86 │      140 │      190 │      260 │      380 │ ▇▇▅▅▃▃ │  │
│ │ adj_clos │  0 │    0 │      210 │       78 │       86 │      140 │      190 │      260 │      380 │ ▇▇▅▅▃▂ │  │
│ │ e        │    │      │          │          │          │          │          │          │          │        │  │
│ │ volume   │  0 │    0 │ 64000000 │ 49000000 │ 10000000 │ 27000000 │ 46000000 │ 95000000 │ 31000000 │  ▇▂▂▁  │  │
│ │          │    │      │          │          │          │          │          │          │        0 │        │  │
│ └──────────┴────┴──────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA     ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ date                   │      0 │         0 │     2023-01-03      │     2023-12-29      │ None             │  │
│ └────────────────────────┴────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ symbol                    │       0 │          0 │  

## Visualize Data

In [ ]:
# plotly engine
df \
.groupby('symbol') \
.plot_timeseries(
    'date', 'adj_close',
    facet_ncol = 2,
    smooth = True,
    smooth_frac = 0.10,
    width = 900,
    height = 700,
    engine = 'plotly'
)

## Technical Indicators

### Trend Indicators:
 - __Moving Averages__ - helps smooth out price data to form a single flowing line, identifying the direction of the trend.

### Add 2 moving averages (20-day and 50-Day)

In [ ]:
sma_df = (
    df[['symbol', 'date', 'adj_close']] 
    .groupby('symbol') 
    .augment_rolling(
        date_column = 'date',
        value_column = 'adj_close',
        window = [20, 50],
        window_func = ['mean'],
        center = False
    )
)

sma_df.head()

,symbol,date,adj_close,adj_close_rolling_mean_win_20,adj_close_rolling_mean_win_50
0,AAPL,2023-01-03,124.216301,NaN,NaN
1,AAPL,2023-01-04,125.497498,NaN,NaN
2,AAPL,2023-01-05,124.166649,NaN,NaN
3,AAPL,2023-01-06,128.735245,NaN,NaN
4,AAPL,2023-01-09,129.261612,NaN,NaN


In [ ]:
# Visualize 
(sma_df 

    # zoom in on dates to filter out `NaN`s
    .query('date >= "2023-06-01"') 

    # Convert to long format
    .melt(
        id_vars = ['symbol', 'date'],
        value_vars = ["adj_close", "adj_close_rolling_mean_win_20", "adj_close_rolling_mean_win_50"]
    ) 

    # Group on symbol and visualize
    .groupby("symbol") 
    .plot_timeseries(
        date_column = 'date',
        value_column = 'value',
        color_column = 'variable',
        smooth = False, 
        facet_ncol = 2,
        width = 900,
        height = 700,
        engine = "plotly"
    )
)

## Returns Analysis

In finance, returns analysis is like checking how well an investment did compared to how much money was put into it. It helps investors see how their investments are doing and decide if they're worth the risk:

- __Performance:__ Returns analysis tells us how well investments are doing and how much risk they carry.
- __Data-driven Decision Making:__ It helps investors and managers make smart choices about where to put their money, how to manage risk, and what investment strategies to follow.

In [ ]:
# Compute returns per each stock
returns_wide_df = (
    df[['symbol', 'date', 'adj_close']]
    .pivot(index = 'date', columns = 'symbol', values = 'adj_close')
    .pct_change()
    .reset_index()[1:]
)

returns_wide_df

symbol,date,AAPL,GOOGL,MSFT,TSLA
1,2023-01-04,0.010314,-0.011670,-0.043743,0.051249
2,2023-01-05,-0.010605,-0.021344,-0.029638,-0.029039
3,2023-01-06,0.036794,0.013225,0.011785,0.024651
4,2023-01-09,0.004089,0.007786,0.009736,0.059349
5,2023-01-10,0.004456,0.004544,0.007617,-0.007681
...,...,...,...,...,...
245,2023-12-22,-0.005547,0.007620,0.002784,-0.007701
246,2023-12-26,-0.002841,0.000212,0.000214,0.016116
247,2023-12-27,0.000518,-0.008126,-0.001575,0.018822
248,2023-12-28,0.002226,-0.000997,0.003235,-0.031594


In [ ]:
# Descriptive Statistics
returns_wide_df.describe(include='number')

symbol,AAPL,GOOGL,MSFT,TSLA
count,249.000000,249.000000,249.000000,249.000000
mean,0.001835,0.001990,0.001972,0.003893
std,0.012570,0.019149,0.015824,0.033166
min,-0.048020,-0.095094,-0.043743,-0.097458
25%,-0.005891,-0.008987,-0.007579,-0.015368
50%,0.001879,0.001006,0.001672,0.003663
75%,0.008846,0.012367,0.011219,0.021004
max,0.046927,0.072787,0.072435,0.110002


In [ ]:
# Correlation
corr_df = (
    returns_wide_df
    .drop('date', axis=1)
    .corr()
)
corr_df

symbol,AAPL,GOOGL,MSFT,TSLA
symbol,,,,
AAPL,1.000000,0.528273,0.547988,0.444279
GOOGL,0.528273,1.000000,0.509887,0.337729
MSFT,0.547988,0.509887,1.000000,0.317910
TSLA,0.444279,0.337729,0.317910,1.000000


## 50-Day Rolling Regression
We need to make a regression function that returns the Slope and Intercept. 

### Get Market Returns
For our purposes, we assume the market is the average returns of the 6 technology stocks.
- We calculate an equal-weight portfolio as the `market returns`.
- Then we merge the market returns into the returns long data.

In [ ]:
# Let’s compute and visualize the 50-day rolling statistics.
returns_long_df = returns_wide_df \
    .melt(id_vars='date', value_name='returns') 

# returns_long_df
rolling_stats_df = (
    returns_long_df.groupby('symbol')
    .augment_rolling(
        date_column = 'date',
        value_column = 'returns',
        window = [50],
        window_func = [
            'mean', 
            'std', 
            'min', 
            ('Q25', lambda x: np.quantile(x,0.25)), 
            'median', 
            ('Q75', lambda x: np.quantile(x, 0.75)), 
            'max'
        ]
    ))

In [ ]:
# Finally, we can .melt() each of the rolling statistics for a Long Format Analysis
rolling_stats_long_df = (
    rolling_stats_df
    .melt(
        id_vars = ["symbol", "date"],
        var_name = "statistic_type"
    )
)

rolling_stats_long_df

,symbol,date,statistic_type,value
0,AAPL,2023-01-04,returns,0.010314
1,AAPL,2023-01-05,returns,-0.010605
2,AAPL,2023-01-06,returns,0.036794
3,AAPL,2023-01-09,returns,0.004089
4,AAPL,2023-01-10,returns,0.004456
...,...,...,...,...
7963,TSLA,2023-12-22,returns_rolling_max_win_50,0.062482
7964,TSLA,2023-12-26,returns_rolling_max_win_50,0.062482
7965,TSLA,2023-12-27,returns_rolling_max_win_50,0.062482
7966,TSLA,2023-12-28,returns_rolling_max_win_50,0.062482


In [ ]:
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetches stock data using yfinance API.

    Args:
    - ticker (str): Stock ticker symbol
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
    - DataFrame: Stock price data
    """
    data = yf.download('BTC-USD', period='1y')
    return data

def create_database(data, db_name):
    """
    Creates an SQLite in-memory database and inserts data.

    Args:
    - data (DataFrame): Stock price data
    - db_name (str): Name of the SQLite in-memory database

    Returns:
    - None
    """
    sql_engine = create_engine(f"sqlite:///{db_name}", echo=False)
    data.to_sql(name='stock_prices', con=sql_engine, if_exists='replace')
    
def transform_to_returns(data):
    """
    Function to transform stock data into daily returns per symbol.
    """
    # Function to transform data into returns
    data['Returns'] = data['Adj Close'].pct_change()
    return data.dropna()

def fetch_data_from_database(db_name):
    """
    Fetches data from the SQLite in-memory database.

    Args:
    - db_name (str): Name of the SQLite in-memory database
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
    - DataFrame: Stock price data
    """
    sql_engine = create_engine(f"sqlite:///{db_name}", echo=False)
    query = "SELECT * FROM stock_prices"
    data = pd.read_sql_query(query, sql_engine)
    
    # Manually set a range of dates as the index
    data.index = pd.date_range(start=start_date, periods=len(data), freq='D')
    
    data = transform_to_returns(data)
    
    return data

In [ ]:
start_date = '2020-01-01'
aapl_df = fetch_data_from_database('stock_data.db') #.info()

In [ ]:
df = aapl_df.reset_index(inplace=False)
df.rename(columns={'index': 'Date', 'Adj Close': 'price'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1005 non-null   datetime64[ns]
 1   Open     1005 non-null   float64       
 2   High     1005 non-null   float64       
 3   Low      1005 non-null   float64       
 4   Close    1005 non-null   float64       
 5   price    1005 non-null   float64       
 6   Volume   1005 non-null   int64         
 7   Returns  1005 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 62.9 KB


In [ ]:
stocks_df = tk.load_dataset('stocks_daily', parse_dates=['date'])
stocks_df.glimpse()

<class 'pandas.core.frame.DataFrame'>: 16194 rows of 8 columns
symbol:    object           ['META', 'META', 'META', 'META', 'META', 'ME ...
date:      datetime64[ns]   [Timestamp('2013-01-02 00:00:00'), Timestamp ...
open:      float64          [27.440000534057617, 27.8799991607666, 28.01 ...
high:      float64          [28.18000030517578, 28.46999931335449, 28.93 ...
low:       float64          [27.420000076293945, 27.59000015258789, 27.8 ...
close:     float64          [28.0, 27.770000457763672, 28.76000022888184 ...
volume:    int64            [69846400, 63140600, 72715400, 83781800, 458 ...
adjusted:  float64          [28.0, 27.770000457763672, 28.76000022888184 ...


In [ ]:
# Plot price series
stocks_df.groupby('symbol').plot_timeseries(
    date_column='date', 
    value_column='adjusted', 
    facet_ncol=3,
    engine='plotly'
)

NameError: name 'stocks_df' is not defined

In [152]:
from sklearn.pipeline import make_pipeline

## 1. Introduction to Time Series Data
- Definition of Time Series Data
- ___Characteristics of Time Series Data:___ Trend, Seasonality, Cyclical Patterns, Random Noise
- Applications of Time Series Analysis in Finance, Economics, and Business

In [10]:
# List of tech companies
tech_companies = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

In [11]:
# Function to download stock data
def download_data(symbol):
    try:
        data = yf.download(symbol, start='2020-01-01', end='2024-01-01')
        return data
    except Exception as e:
        print(f"Error downloading data for {symbol}: {e}")
        return None

In [12]:
def transform_to_returns(data):
    """
    Function to transform stock data into daily returns per symbol.
    """
    # Function to transform data into returns
    data['Returns'] = data['Adj Close'].pct_change()
    return data.dropna()

In [13]:
# Function to load data into SQLite database
def load_data(symbol, data):
    """
    Function to load transformed data into an `tech_stocks.db` SQLite database.
    """
    conn = sqlite3.connect('tech_stocks.db')
    data.to_sql(name=symbol, con=conn, if_exists='replace', index=False)
    conn.close()

In [17]:
# ETL process
def etl_process():
    for symbol in tech_companies:
        print(f"Downloading data for {symbol}...")
        stock_data = download_data(symbol)
        if stock_data is not None:
            print(f"Calculating returns for {symbol}...")
            returns_data = transform_to_returns(stock_data)
            print(f'>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>')
            load_data(symbol, returns_data)
    
    print(f"{'='}"*30)
    print(f'ETL Process is Complete!')

In [18]:
if __name__ == "__main__":
    etl_process()

[*********************100%%**********************]  1 of 1 completed
Calculating returns for AAPL...
>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>
[*********************100%%**********************]  1 of 1 completed
Calculating returns for MSFT...
>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>
[*********************100%%**********************]  1 of 1 completed
Calculating returns for GOOGL...
>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>
[*********************100%%**********************]  1 of 1 completed
Calculating returns for AMZN...
>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>
[*********************100%%**********************]  1 of 1 completed
Calculating returns for TSLA...
>>>>>>>>>>>>>>>>DONE>>>>>>>>>>>>>>>
ETL Process is Complete!


In [19]:
def get_table_names(db_name='tech_stocks.db'):
    """
    Function to retrieve table names from the SQLite database.
    Returns a list of table names.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    query = """
            SELECT name FROM sqlite_master WHERE type='table'
            """
    cursor.execute(query)
    table_names = [table[0] for table in cursor.fetchall()]
    conn.close()
    return table_names

'NVDA'

In [22]:
def fetch_from_database(symbol, db_name='tech_stocks.db'):
    """
    Function to fetch data for a specific stock symbol from SQLite database into a pandas dataframe.
    """
    conn = sqlite3.connect(db_name)
    query = f"SELECT * FROM {symbol}"
    df = pd.read_sql_query(query, conn, index_col='date')
    conn.close()
    return df

In [23]:
get_table_names()

['FB', 'NVDA', 'INTC', 'IBM', 'ORCL', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

In [24]:
fetch_from_database(get_table_names()[1])

KeyError: "None of ['date'] are in the columns"

In [173]:
def main():
    # Download stock data
    print("Downloading stock data...")
    stock_data = download_stock_data(tech_companies)
    print("All stock data downloaded successfully.")
    
    # Convert data into dataframes
    for symbol, data in stock_data.items():
        stock_data[symbol] = pd.DataFrame(data)

    # Transform data into returns
    print("Transforming data into returns...")
    returns_data = transform_to_returns(stock_data)
    print("Data transformed successfully.")

    # Load data into database
    print("Loading data into database...")
    load_into_database(returns_data)
    print("Data loaded into database successfully.")
    
    # Fetch data for the specified symbol
    
    # Symbol to fetch data for
    symbol_to_fetch = 'AAPL'  # Example symbol

    print(f"Fetching data for {symbol_to_fetch} from database...")
    df = fetch_from_database(symbol_to_fetch)
    print("Data fetched successfully.")

    # Display the dataframe
    print("DataFrame for", symbol_to_fetch)
    print(df.head())  # Displaying first few rows of data

In [174]:
if __name__ == "__main__":
    main()

[*********************100%%**********************]  1 of 1 completed
AAPL stock data downloaded.
------------------------------
[*********************100%%**********************]  1 of 1 completed
MSFT stock data downloaded.
------------------------------
[*********************100%%**********************]  1 of 1 completed
GOOGL stock data downloaded.
------------------------------
[*********************100%%**********************]  1 of 1 completed
AMZN stock data downloaded.
------------------------------
[*********************100%%**********************]  1 of 1 completed
TSLA stock data downloaded.
------------------------------
All stock data downloaded successfully.
Transforming data into returns...


KeyError: 'Adj Close'

## 2. Time Series Decomposition
- Understanding Trend, Seasonality, and Residual Components
- ___Decomposition Techniques:___ Additive Decomposition, Multiplicative Decomposition
- Implementing Decomposition Methods using Python libraries like statsmodels

## 3. Stationarity and Differencing
- ___Stationarity:___ Definition and Importance in Time Series Analysis
- Identifying Stationarity using Statistical Tests (e.g., Augmented Dickey-Fuller test)
- ___Techniques for Achieving Stationarity:___ Differencing, Transformation
- Implementing Stationarity Checks and Differencing in Python with pandas and numpy

## 4. Autoregressive Integrated Moving Average (ARIMA) Model
- ___Introduction to ARIMA Model and its Components:___ Autoregressive (AR), Integrated (I), Moving Average (MA)
- ___Understanding the Parameters of ARIMA Model:___ p, d, q
- ___Steps for Building an ARIMA Model:___ Model Identification, Parameter Estimation, Model Diagnostic Checking, Forecasting
- Implementing ARIMA Modeling in Python with statsmodels library

## 5. Model Evaluation Metrics
- ___Mean Absolute Percentage Error (MAPE):___ Interpretation and Calculation
- ___Root Mean Squared Error (RMSE):___ Interpretation and Calculation
- Comparison of Different Models based on Evaluation Metrics
- ___Hands-on Practice:___ Evaluating ARIMA Model Performance on Stock Price Data

## 6. Advanced Time Series Techniques (Optional)
- ___Seasonal ARIMA (SARIMA) Model:___ Incorporating Seasonal Components into ARIMA
- ___Exponential Smoothing Methods:___ Simple Exponential Smoothing (SES), Holt-Winters Exponential Smoothing
- ___Prophet:___ Facebook's Time Series Forecasting Tool
- Introduction to Deep Learning-based Time Series Forecasting Models (e.g., LSTM)

## 7. Applications in Stock Price Prediction
- Forecasting Stock Prices using ARIMA Model
- Interpreting Model Forecasts and Understanding Limitations
- Incorporating External Factors for Improved Forecasting Accuracy
- Real-world Case Studies and Examples of Time Series Analysis in Financial Markets

## 8. Project: Stock Price Prediction

___Hands-on Project:___ Building a Stock Price Prediction Model using Time Series Analysis Techniques
- Data Collection and Preprocessing
- Model Building and Evaluation
- Fine-tuning Model Parameters for Improved Performance
- Presenting and Interpreting Results

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import yfinance as yf

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# Define the layout
app.layout = dbc.Container([
    html.H1("Portfolio Risk Analysis", className="mt-4"),
    dbc.Row([
        dbc.Col([
            html.Label("Enter stock symbols (comma-separated):"),
            dcc.Input(id="stock-input", type="text", value="AAPL,GOOGL,MSFT"),
            html.Label("Select risk metric:"),
            dcc.Dropdown(
                id="risk-metric",
                options=[
                    {'label': 'Standard Deviation', 'value': 'std_dev'},
                    {'label': 'Beta', 'value': 'beta'}
                ],
                value='std_dev'
            ),
        ], md=4),
        dbc.Col([
            dcc.Graph(id="risk-return-plot")
        ], md=8)
    ])
], fluid=True)

# Define callback to update the plot
@app.callback(
    Output("risk-return-plot", "figure"),
    [Input("stock-input", "value"),
     Input("risk-metric", "value")]
)
def update_plot(stock_symbols, risk_metric):
    # Retrieve stock data
    stocks = yf.download(stock_symbols.split(','), period="1y")['Adj Close']
    
    # Calculate risk metric
    if risk_metric == 'std_dev':
        risk_data = stocks.pct_change().std()
    elif risk_metric == 'beta':
        # Calculate beta using market data
        market_data = yf.download('SPY', period="1y")['Adj Close'].pct_change().dropna()
        risk_data = stocks.pct_change().cov(market_data) / market_data.var()
    
    # Plotting
    fig = {
        'data': [
            {'x': risk_data, 'y': stocks.pct_change().mean(), 'type': 'scatter', 'mode': 'markers'}
        ],
        'layout': {
            'title': 'Risk vs Return',
            'xaxis': {'title': risk_metric},
            'yaxis': {'title': 'Mean Return'}
        }
    }
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port='8000')